In [1]:
%matplotlib inline
%pylab inline
import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [3]:
import os
import random
import skimage.data
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Allow image embeding in notebook
%matplotlib inline

In [4]:
def load_data(data_dir):
    """Loads a data set and returns two lists:
    
    images: a list of Numpy arrays, each representing an image.
    labels: a list of numbers that represent the images labels.
    """
    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) 
                      for f in os.listdir(label_dir) if f.endswith(".ppm")]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    return images, labels

In [5]:
ROOT_PATH = "./data/"
data_dir = os.path.join(ROOT_PATH, "speed-limit-signs")

images, labels = load_data(data_dir)

In [99]:
images128 = [skimage.transform.resize(image, (128,128))
                for image in images]

In [100]:
y = np.array(labels)
X = np.array(images128)

In [16]:
y.shape

(379,)

In [96]:
X.shape

(379, 100, 100, 3)

In [98]:
a = Input(shape=(3, 32, 32))

In [87]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Input
from keras.optimizers import SGD

In [93]:
a = Input(shape=(3, 32, 32))
b = Input(shape=(3, 64, 64))

conv = Convolution2D(16, 3, 3, border_mode='same')
conved_a = conv(a)
conv.output_shape

(None, 3, 32, 16)

In [94]:
conv.input_shape

(None, 3, 32, 32)

In [95]:

# only one input so far, the following will work:
#assert conv.input_shape == (None, 3, 32, 32)

conved_b = conv(b)

ValueError: Dimensions must be equal, but are 64 and 32 for 'Conv2D_77' (op: 'Conv2D') with input shapes: [?,3,64,64], [3,3,32,16].

In [78]:
model = Sequential()
# input: 128x128 images with 3 channels -> (3, 128, 128) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=(3, 128, 128)))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3, border_mode='same'))
model.add(Activation('relu'))

In [79]:
model.output_shape

(None, 3, 128, 32)

In [80]:
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [81]:
model.output_shape

(None, 1, 64, 32)

In [82]:
model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))

In [83]:
model.output_shape

(None, 1, 64, 64)

In [86]:
model.add(MaxPooling2D(pool_size=(2, 2))
model.add(Dropout(0.25))

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'MaxPool_13' (op: 'MaxPool') with input shapes: [?,1,64,64].

In [ ]:

model.add(Flatten())
# Note: Keras does automatic shape inference.
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(6))
model.add(Activation('softmax'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

In [ ]:
model.fit(X, y, batch_size=32, nb_epoch=1)